In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

import sys
sys.path.append('c:/Users/tomas/Documents/Programación/Github/Patricionog/Modulio/Readmes')
import Appio as ap
import Archivio as ac
import Databasio as bd
import Datetimio as dt
import Dictio as dc
import Excelio as ex
import Footio as ft
import Foragio as fg
import Framio as fr
import Graphio as gr
import Highlightio as hg
import Inputio as ip
import Listio as ls
import Mathio as mt
import Notio as nt
import Numbio as nb
import Numpio as nm
import Stringio as st
import Timio as tm
import Tkintio as tn
import Toolio as to

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.miscmodels.ordinal_model import OrderedModel
from statsmodels.stats.proportion import proportions_chisquare
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy import stats

Los métodos robustos disminuyen el grado en que los resultados pueden verse influenciados por violaciones de los supuestos estadísticos clásicos.

In [2]:
# Cargar df.
df = pd.read_excel('J:/My Drive/Experimento/Plantillas finales/1. Generales - Original.xlsx')  

In [3]:
# Definir variables.
Variables_Independientes = ['genero',
            'edad',
            #'nacionalidad',
            #'provincia',
            #'e_social',
            #'niv_educativo',
            'voto_2019',
            #'voto_PASO_2023',
            #'candidato_PASO_2023',
            'autopercep_izqder',
            'autopercep_conpro',
            'autopercep_perantiper',
            'indice_positividad',
            #'massa_ip_izqder',
            #'massa_ip_conpro',
            #'bullrich_ip_izqder',
            #'bullrich_ip_conpro',
            #'schiaretti_ip_izqder',
            #'schiaretti_ip_conpro',
            #'milei_ip_izqder',
            #'milei_ip_conpro',
            #'bregman_ip_izqder',
            #'bregman_ip_conpro',
            'indice_progresismo',
            'indice_conservadurismo'
            ]

Variables_Dependientes = ['cercania_Massa', 'cercania_Milei', 
                          'cercania_Bregman', 'cercania_Bullrich', 'cercania_Schiaretti']

In [4]:
# Rellenar valores NaN con 0 para todo el DataFrame.
df.fillna(0, inplace=True)

In [5]:
# Cortar df.
df = df[Variables_Dependientes + Variables_Independientes]

In [6]:
# Convertir variables categóricas a dummies.
for Column in Variables_Independientes:
    if df[Column].dtype == 'object':
        df = fr.Create_Dummy_Variables(df, Column, Drop_First = False, Group_Others = True, Remove_Others = False, 
                                    Threshold = 0.05, Name_Other_Column = f"{Column}_Others", Name_Columns_Style = None, Separator=None)

In [7]:
# Reemplazar espacios en los nombres de las columnas con guiones bajos
df.columns = df.columns.str.replace(' ', '_')

df

,cercania_Massa,cercania_Milei,cercania_Bregman,cercania_Bullrich,cercania_Schiaretti,edad,autopercep_izqder,autopercep_conpro,autopercep_perantiper,indice_positividad,indice_progresismo,indice_conservadurismo,genero_femenino,genero_masculino,genero_Others,voto_2019_Alberto_Fernandez,voto_2019_Mauricio_Macri,voto_2019_Nicolás_Del_Caño,voto_2019_no_vote,voto_2019_Others
0,1,2,1,4,5,67,6,8,10,15,2.000000,4.111111,True,False,0,False,True,False,False,0
1,1,3,1,1,4,54,8,2,3,15,3.555556,2.700000,False,True,0,False,False,False,False,1
2,1,1,1,5,1,68,10,10,10,15,3.000000,2.600000,False,True,0,False,True,False,False,0
3,1,1,1,3,1,50,10,2,10,13,2.800000,3.444444,True,False,0,False,True,False,False,0
4,1,3,1,5,3,37,6,5,6,13,2.400000,4.000000,True,False,0,False,True,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2834,1,5,1,3,3,45,10,10,10,16,3.600000,2.777778,False,True,0,False,True,False,False,0
2835,5,1,3,1,3,24,4,8,1,18,3.000000,4.333333,False,True,0,True,False,False,False,0
2836,1,5,1,3,5,40,10,1,10,11,3.444444,1.700000,True,False,0,False,False,False,False,1
2837,1,1,1,1,1,53,6,1,10,11,4.111111,2.700000,True,False,0,False,False,False,False,1


In [8]:
# Object to numeric df.
def Convert_Object_Columns_In_Numerics(df):
    # Convertir columnas booleanas a numéricas (0 y 1).
    for Column in df.columns:
        if df[Column].dtype == 'bool':
            df[Column] = df[Column].astype(int)
    
    return df

df = Convert_Object_Columns_In_Numerics(df)

In [9]:
# Redefinir las variables independientes.
Variables_Independientes = [Column for Column in df.columns if Column not in Variables_Dependientes]
Variables_Independientes.remove('genero_Others')
Variables_Independientes.remove('genero_masculino')
Variables_Independientes.remove('voto_2019_Others')
Variables_Independientes.remove('voto_2019_no_vote')
Variables_Independientes.remove('voto_2019_Nicolás_Del_Caño')

In [10]:
# Escalar variables independientes.
from sklearn.preprocessing import StandardScaler

Scaler = StandardScaler()
df[Variables_Independientes] = Scaler.fit_transform(df[Variables_Independientes])

In [11]:
def Generalized_Ordinal_Regression(df: pd.DataFrame, V_Ind: list, V_Dep: str, Summary: bool = False) -> tuple:
    """
    Perform an ordinal regression using OrderedModel from statsmodels.
    
    Args:
        df (pd.DataFrame): The dataframe containing the data.
        V_Ind (list): List of independent variable names.
        V_Dep (str): Name of the dependent variable.
        Summary (bool): Whether to print the summary of the regression model.
        
    Returns:
        tuple: A tuple containing:
            - A list of significant variables based on p-values.
            - A DataFrame with p-values and odds ratios for each variable.
    """
    
    # Define the dependent and independent variables.
    Y = df[V_Dep]
    X = df[V_Ind]

    # Fit the ordinal regression model.
    Model = OrderedModel(Y, X, distr='logit')  # You can use 'probit' instead of 'logit' if preferred.
    Results = Model.fit(method='bfgs', maxiter=1000)

    # Print the summary if requested.
    if Summary:
        print(Results.summary())

    # Generate a list of variables with p-value less than 0.05.
    Significant_Variables = Results.pvalues[Results.pvalues < 0.05].index.tolist()

    # Calculate odds ratios.
    Odds_Ratios = np.exp(Results.params)  # Calculate the exponentiated coefficients.

    # Create a DataFrame to hold p-values and odds ratios.
    Summary_Statistics = pd.DataFrame({
        'Variable': Results.params.index,
        'P_Value': Results.pvalues,
        'Odds_Ratio': Odds_Ratios
    })

    return Significant_Variables, Summary_Statistics

In [12]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def Check_Multicollinearity(X: pd.DataFrame):

    """Calculate VIF for each feature in the DataFrame and return variables with high multicollinearity."""
    
    VIF_Data = pd.DataFrame()
    VIF_Data["Feature"] = X.columns
    VIF_Data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    # Display features with VIF > 10 (considered problematic)
    print(VIF_Data[VIF_Data["VIF"] > 10])
    
Check_Multicollinearity(df[Variables_Independientes])

Empty DataFrame
Columns: [Feature, VIF]
Index: []


In [13]:
for Variable_Dependiente in Variables_Dependientes: 
    V_Dependientes_Para_Comparar = Variables_Independientes

    print('----------------------------------------------------------------')
    print(f'Variable dependiente: {Variable_Dependiente}.')
    print('----------------------------------------------------------------')

    Significativas, Summary_Statistics = Generalized_Ordinal_Regression(df, Variables_Independientes, Variable_Dependiente)

    Variables_To_Remove = ['const', '1/2', '2/3', '3/4', '4/5']
    for Variable in Variables_To_Remove:
        if Variable in Significativas:
            Significativas.remove(Variable)

    # Check assumptions and remove insignificant variables iteratively.
    while sorted(Significativas) != sorted(V_Dependientes_Para_Comparar):
        V_Dependientes_Para_Comparar = Significativas
        Significativas, Summary_Statistics = Generalized_Ordinal_Regression(df, V_Dependientes_Para_Comparar, Variable_Dependiente)

        for Variable in Variables_To_Remove:
            if Variable in Significativas:
                Significativas.remove(Variable)
    
    print(f'Variables significativas:')
    for Variable in Significativas:
        Odds_Ratio_Value = Summary_Statistics.loc[Summary_Statistics['Variable'] == Variable, 'Odds_Ratio'].values[0]
        P_Value = Summary_Statistics.loc[Summary_Statistics['Variable'] == Variable, 'P_Value'].values[0]
        print(f'Variable: {Variable}, Odds Ratio: {Odds_Ratio_Value:.4f}, P-Value: {P_Value:.4f}')

----------------------------------------------------------------
Variable dependiente: cercania_Massa.
----------------------------------------------------------------
Optimization terminated successfully.
         Current function value: 1.158434
         Iterations: 26
         Function evaluations: 27
         Gradient evaluations: 27
Optimization terminated successfully.
         Current function value: 1.159259
         Iterations: 25
         Function evaluations: 26
         Gradient evaluations: 26
Variables significativas:
Variable: edad, Odds Ratio: 0.8709, P-Value: 0.0003
Variable: autopercep_perantiper, Odds Ratio: 0.4062, P-Value: 0.0000
Variable: indice_positividad, Odds Ratio: 1.7151, P-Value: 0.0000
Variable: indice_progresismo, Odds Ratio: 0.8238, P-Value: 0.0003
Variable: indice_conservadurismo, Odds Ratio: 1.4206, P-Value: 0.0000
Variable: voto_2019_Alberto_Fernandez, Odds Ratio: 1.8332, P-Value: 0.0000
Variable: voto_2019_Mauricio_Macri, Odds Ratio: 0.8268, P-Value:

In [14]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

def Chequeo_Proporcionalidad_Logitos(DataFrame: pd.DataFrame, Variable_Ordinal: str, Variables_Independientes: list) -> None:
    """
    Check the proportional odds assumption for an ordinal regression model.
    
    Args:
        DataFrame (pd.DataFrame): The DataFrame containing the data.
        Variable_Ordinal (str): The ordinal dependent variable name.
        Variables_Independientes (list): List of independent variable names.
    
    Returns:
        None
    """
    
    # Check that the dependent variable and independent variables are in the DataFrame
    if Variable_Ordinal not in DataFrame.columns:
        print(f"Error: La variable ordinal '{Variable_Ordinal}' no está en el DataFrame.")
        return
    
    for Variable in Variables_Independientes:
        if Variable not in DataFrame.columns:
            print(f"Error: La variable independiente '{Variable}' no está en el DataFrame.")
            return
    
    # Fit the ordinal model
    Formula = f"{Variable_Ordinal} ~ " + " + ".join(Variables_Independientes)
    Model = smf.mnlogit(formula=Formula, data=DataFrame).fit()
    
    # Get the coefficients
    Coeficientes = Model.params
    
    # Display the coefficients
    print("Coeficientes del modelo por categoría:")
    print(Coeficientes)

    # Check proportionality assumption
    # Get the categories from the coefficients index
    Categorias = Coeficientes.index.unique(level=0).tolist()
    
    # Compare coefficients between categories
    for i in range(1, len(Categorias)):
        Coef1 = Coeficientes.loc[Categorias[i]]
        Coef0 = Coeficientes.loc[Categorias[0]]
        
        Igualdad = np.all(np.isclose(Coef1, Coef0))
        
        if Igualdad:
            print(f"Se cumple el supuesto de proporcionalidad de los logitos para la categoría '{Categorias[i]}'.")
        else:
            print(f"No se cumple el supuesto de proporcionalidad de los logitos para la categoría '{Categorias[i]}'.")
            
            
# Uso de la función
# Reemplaza 'tu_dataframe', 'variable_ordinal', y 'variables_independientes' con tus datos
Chequeo_Proporcionalidad_Logitos(df, 'cercania_Massa', Variables_Independientes)


Optimization terminated successfully.
         Current function value: 1.143257
         Iterations 7
Coeficientes del modelo por categoría:
                                    0         1         2         3
Intercept                   -0.291514  0.464030 -0.436612 -1.637047
edad                        -0.051833 -0.037489 -0.257072 -0.207692
autopercep_izqder           -0.052944  0.038178  0.289458  0.253393
autopercep_conpro            0.192909  0.153529  0.196321  0.031149
autopercep_perantiper       -0.354306 -0.778950 -1.338563 -1.958640
indice_positividad           0.187117  0.598948  0.824476  1.204713
indice_progresismo          -0.101333 -0.335761 -0.374867 -0.592645
indice_conservadurismo       0.374513  0.522485  0.718888  0.459055
genero_femenino              0.019523  0.047025  0.109325  0.123661
voto_2019_Alberto_Fernandez  0.461804  0.764314  1.031303  1.061383
voto_2019_Mauricio_Macri    -0.171460 -0.122634 -0.320538 -0.255857
No se cumple el supuesto de proporcionalida